In [ ]:
import numpy as np
import xarray as xr
import os,errno
import sys
import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt
import salem
import scipy.stats as st

dir_data='/Users/ahenny/'
dir2='/Volumes/Extreme Pro/'
dir3='/Volumes/My Passport/'

In [ ]:
yrs=np.arange(1940,2024,1)

ds=xr.open_dataset(dir2+'merra2_gridareas.nc')
areas=ds['cell_area']/1e6

nums_array_era5_polar=[[],[],[],[],[],[],[],[],[],[],[],[]]
areas_array_era5_polar=[[],[],[],[],[],[],[],[],[],[],[],[]]
lats_array_era5_polar=[[],[],[],[],[],[],[],[],[],[],[],[]]
nums_array_era5_fixed=[[],[],[],[],[],[],[],[],[],[],[],[]]
areas_array_era5_fixed=[[],[],[],[],[],[],[],[],[],[],[],[]]
lats_array_era5_fixed=[[],[],[],[],[],[],[],[],[],[],[],[]]
nums_array_era5_fixed_poleward=[[],[],[],[],[],[],[],[],[],[],[],[]]
areas_array_era5_fixed_poleward=[[],[],[],[],[],[],[],[],[],[],[],[]]
lats_array_era5_fixed_poleward=[[],[],[],[],[],[],[],[],[],[],[],[]]

lats_array_era5_polar_nh=[[],[],[],[],[],[],[],[],[],[],[],[]]
lats_array_era5_fixed_nh=[[],[],[],[],[],[],[],[],[],[],[],[]]
lats_array_era5_fixed_poleward_nh=[[],[],[],[],[],[],[],[],[],[],[],[]]
lats_array_era5_polar_sh=[[],[],[],[],[],[],[],[],[],[],[],[]]
lats_array_era5_fixed_sh=[[],[],[],[],[],[],[],[],[],[],[],[]]
lats_array_era5_fixed_poleward_sh=[[],[],[],[],[],[],[],[],[],[],[],[]]

nums_list_era5_polar=[]
areas_list_era5_polar=[]
lats_list_era5_polar=[]
nums_list_era5_fixed=[]
areas_list_era5_fixed=[]
lats_list_era5_fixed=[]
nums_list_era5_fixed_poleward=[]
areas_list_era5_fixed_poleward=[]
lats_list_era5_fixed_poleward=[]

lats_list_era5_polar_nh=[]
lats_list_era5_fixed_nh=[]
lats_list_era5_fixed_poleward_nh=[]
lats_list_era5_polar_sh=[]
lats_list_era5_fixed_sh=[]
lats_list_era5_fixed_poleward_sh=[]

for i in range(len(yrs)):
    year=yrs[i]
    print(year)
    
    #################
    ds=xr.open_dataset(dir2+'era5.ar.labels.model.'+str(year)+'.nc')
    labeled=ds['ar_labeled']
    
    if i==0:
        lat_array=xr.zeros_like(labeled[0,:,:])
        lon_array=xr.zeros_like(labeled[0,:,:])
        for k in range(labeled.lon.size):
            lat_array[:,k]=labeled.lat.values
        for k in range(labeled.lat.size):
            lon_array[k,:]=labeled.lon.values
            
        weights=np.cos(np.deg2rad(ds.lat))
    
    num_list_year=[]
    areas_list_year=[]
    lats_list_year=[]
    lats_list_year_nh=[]
    lats_list_year_sh=[]
    
    nums_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
    areas_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
    lats_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
    lats_array_year_nh=[[],[],[],[],[],[],[],[],[],[],[],[]]
    lats_array_year_sh=[[],[],[],[],[],[],[],[],[],[],[],[]]

    count=0
    for j in range(labeled.time.size):
        date=pd.to_datetime(labeled.time.values[j])
        month=date.month
        
        label_sel=labeled[j,:,:]
        unique=np.unique(label_sel.values)
        unique=[x for x in unique if x>0]
        num=len(unique)
        
        num_day=0
        for k in range(len(unique)):
            sel_feature=label_sel.where(label_sel==unique[k])
            area_feature=areas.where(sel_feature>0)
            area_feature=area_feature.sum(skipna=True).values.tolist()
            lats_sel=lat_array.where(label_sel==unique[k])
            lats_weighted=lats_sel.weighted(weights)
            lat_mean=lats_weighted.mean(skipna=True).values.tolist()
            
            if area_feature>0:
                areas_list_year.append(area_feature)
                lats_list_year.append(abs(lat_mean))
                areas_array_year[month-1].append(area_feature)
                lats_array_year[month-1].append(abs(lat_mean))
                
                if lat_mean>0:
                    lats_list_year_nh.append(lat_mean)
                    lats_array_year_nh[month-1].append(lat_mean)
                        
                if lat_mean<0:
                    lats_list_year_sh.append(lat_mean)
                    lats_array_year_sh[month-1].append(lat_mean)
                        
                num_day=num_day+1
    
        num_list_year.append(num_day)
        nums_array_year[month-1].append(num_day)
        
    mean_num=float(sum(num_list_year))/float(len(num_list_year))
    mean_area=float(sum(areas_list_year))/float(len(areas_list_year))
    mean_lat=float(sum(lats_list_year))/float(len(lats_list_year))
    nums_list_era5_fixed.append(mean_num)
    areas_list_era5_fixed.append(mean_area)
    lats_list_era5_fixed.append(mean_lat)
    
    mean_lat_nh=float(sum(lats_list_year_nh))/float(len(lats_list_year_nh))
    mean_lat_sh=float(sum(lats_list_year_sh))/float(len(lats_list_year_sh))
    lats_list_era5_fixed_nh.append(mean_lat_nh)
    lats_list_era5_fixed_sh.append(mean_lat_sh)
    
    for j in range(len(nums_array_year)):
        nums_array_era5_fixed[j].append(float(sum(nums_array_year[j]))/float(len(nums_array_year[j])))#mean of all time step AR numbers
        areas_array_era5_fixed[j].append(float(sum(areas_array_year[j]))/float(len(areas_array_year[j])))#mean of all areas in month
        lats_array_era5_fixed[j].append(float(sum(lats_array_year[j]))/float(len(lats_array_year[j])))#mean of all avg lats in month
        lats_array_era5_fixed_nh[j].append(float(sum(lats_array_year_nh[j]))/float(len(lats_array_year_nh[j])))#mean of all avg lats in month
        lats_array_era5_fixed_sh[j].append(float(sum(lats_array_year_sh[j]))/float(len(lats_array_year_sh[j])))#mean of all avg lats in month

        
    ################
    
    ds=xr.open_dataset(dir2+'era5.ar.labels.model.variable.'+str(year)+'.nc')
    labeled=ds['ar_labeled']
    
    if i==0:
        lat_array=xr.zeros_like(labeled[0,:,:])
        lon_array=xr.zeros_like(labeled[0,:,:])
        for k in range(labeled.lon.size):
            lat_array[:,k]=labeled.lat.values
        for k in range(labeled.lat.size):
            lon_array[k,:]=labeled.lon.values
            
        weights=np.cos(np.deg2rad(ds.lat))
    
    num_list_year=[]
    areas_list_year=[]
    lats_list_year=[]
    lats_list_year_nh=[]
    lats_list_year_sh=[]
    
    nums_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
    areas_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
    lats_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
    lats_array_year_nh=[[],[],[],[],[],[],[],[],[],[],[],[]]
    lats_array_year_sh=[[],[],[],[],[],[],[],[],[],[],[],[]]

    count=0
    for j in range(labeled.time.size):
        date=pd.to_datetime(labeled.time.values[j])
        month=date.month
        
        label_sel=labeled[j,:,:]
        unique=np.unique(label_sel.values)
        unique=[x for x in unique if x>0]
        num=len(unique)
        
        num_day=0
        for k in range(len(unique)):
            sel_feature=label_sel.where(label_sel==unique[k])
            area_feature=areas.where(sel_feature>0)
            area_feature=area_feature.sum(skipna=True).values.tolist()
            lats_sel=lat_array.where(label_sel==unique[k])
            lats_weighted=lats_sel.weighted(weights)
            lat_mean=lats_weighted.mean(skipna=True).values.tolist()
            
            if area_feature>0:
                areas_list_year.append(area_feature)
                lats_list_year.append(abs(lat_mean))
                areas_array_year[month-1].append(area_feature)
                lats_array_year[month-1].append(abs(lat_mean))
                
                if lat_mean>0:
                    lats_list_year_nh.append(lat_mean)
                    lats_array_year_nh[month-1].append(lat_mean)
                        
                if lat_mean<0:
                    lats_list_year_sh.append(lat_mean)
                    lats_array_year_sh[month-1].append(lat_mean)
                        
                num_day=num_day+1
    
        num_list_year.append(num_day)
        nums_array_year[month-1].append(num_day)
        
    mean_num=float(sum(num_list_year))/float(len(num_list_year))
    mean_area=float(sum(areas_list_year))/float(len(areas_list_year))
    mean_lat=float(sum(lats_list_year))/float(len(lats_list_year))
    nums_list_era5_polar.append(mean_num)
    areas_list_era5_polar.append(mean_area)
    lats_list_era5_polar.append(mean_lat)
    
    mean_lat_nh=float(sum(lats_list_year_nh))/float(len(lats_list_year_nh))
    mean_lat_sh=float(sum(lats_list_year_sh))/float(len(lats_list_year_sh))
    lats_list_era5_polar_nh.append(mean_lat_nh)
    lats_list_era5_polar_sh.append(mean_lat_sh)
    
    for j in range(len(nums_array_year)):
        nums_array_era5_polar[j].append(float(sum(nums_array_year[j]))/float(len(nums_array_year[j])))#mean of all time step AR numbers
        areas_array_era5_polar[j].append(float(sum(areas_array_year[j]))/float(len(areas_array_year[j])))#mean of all areas in month
        lats_array_era5_polar[j].append(float(sum(lats_array_year[j]))/float(len(lats_array_year[j])))#mean of all avg lats in month
        lats_array_era5_polar_nh[j].append(float(sum(lats_array_year_nh[j]))/float(len(lats_array_year_nh[j])))#mean of all avg lats in month
        lats_array_era5_polar_sh[j].append(float(sum(lats_array_year_sh[j]))/float(len(lats_array_year_sh[j])))#mean of all avg lats in month

    #################
    ds=xr.open_dataset(dir2+'era5.ar.labels.model.poleward.'+str(year)+'.nc')
    labeled=ds['ar_labeled']
    
    if i==0:
        lat_array=xr.zeros_like(labeled[0,:,:])
        lon_array=xr.zeros_like(labeled[0,:,:])
        for k in range(labeled.lon.size):
            lat_array[:,k]=labeled.lat.values
        for k in range(labeled.lat.size):
            lon_array[k,:]=labeled.lon.values
            
        weights=np.cos(np.deg2rad(ds.lat))
    
    num_list_year=[]
    areas_list_year=[]
    lats_list_year=[]
    lats_list_year_nh=[]
    lats_list_year_sh=[]
    
    nums_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
    areas_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
    lats_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
    lats_array_year_nh=[[],[],[],[],[],[],[],[],[],[],[],[]]
    lats_array_year_sh=[[],[],[],[],[],[],[],[],[],[],[],[]]

    count=0
    for j in range(labeled.time.size):
        date=pd.to_datetime(labeled.time.values[j])
        month=date.month
        
        label_sel=labeled[j,:,:]
        unique=np.unique(label_sel.values)
        unique=[x for x in unique if x>0]
        num=len(unique)
        
        num_day=0
        for k in range(len(unique)):
            sel_feature=label_sel.where(label_sel==unique[k])
            area_feature=areas.where(sel_feature>0)
            area_feature=area_feature.sum(skipna=True).values.tolist()
            lats_sel=lat_array.where(label_sel==unique[k])
            lats_weighted=lats_sel.weighted(weights)
            lat_mean=lats_weighted.mean(skipna=True).values.tolist()
            
            if area_feature>0:
                areas_list_year.append(area_feature)
                lats_list_year.append(abs(lat_mean))
                areas_array_year[month-1].append(area_feature)
                lats_array_year[month-1].append(abs(lat_mean))
                
                if lat_mean>0:
                    lats_list_year_nh.append(lat_mean)
                    lats_array_year_nh[month-1].append(lat_mean)
                        
                if lat_mean<0:
                    lats_list_year_sh.append(lat_mean)
                    lats_array_year_sh[month-1].append(lat_mean)
                        
                num_day=num_day+1
    
        num_list_year.append(num_day)
        nums_array_year[month-1].append(num_day)
        
    mean_num=float(sum(num_list_year))/float(len(num_list_year))
    mean_area=float(sum(areas_list_year))/float(len(areas_list_year))
    mean_lat=float(sum(lats_list_year))/float(len(lats_list_year))
    nums_list_era5_fixed_poleward.append(mean_num)
    areas_list_era5_fixed_poleward.append(mean_area)
    lats_list_era5_fixed_poleward.append(mean_lat)
    
    mean_lat_nh=float(sum(lats_list_year_nh))/float(len(lats_list_year_nh))
    mean_lat_sh=float(sum(lats_list_year_sh))/float(len(lats_list_year_sh))
    lats_list_era5_fixed_poleward_nh.append(mean_lat_nh)
    lats_list_era5_fixed_poleward_sh.append(mean_lat_sh)
    
    for j in range(len(nums_array_year)):
        nums_array_era5_fixed_poleward[j].append(float(sum(nums_array_year[j]))/float(len(nums_array_year[j])))#mean of all time step AR numbers
        areas_array_era5_fixed_poleward[j].append(float(sum(areas_array_year[j]))/float(len(areas_array_year[j])))#mean of all areas in month
        lats_array_era5_fixed_poleward[j].append(float(sum(lats_array_year[j]))/float(len(lats_array_year[j])))#mean of all avg lats in month
        lats_array_era5_fixed_poleward_nh[j].append(float(sum(lats_array_year_nh[j]))/float(len(lats_array_year_nh[j])))#mean of all avg lats in month
        lats_array_era5_fixed_poleward_sh[j].append(float(sum(lats_array_year_sh[j]))/float(len(lats_array_year_sh[j])))#mean of all avg lats in month


In [ ]:
dk=xr.Dataset()
dk['nums_array_era5_polar']=(('months','years'),np.asarray(nums_array_era5_polar))
dk['areas_array_era5_polar']=(('months','years'),np.asarray(areas_array_era5_polar))
dk['lats_array_era5_polar']=(('months','years'),np.asarray(lats_array_era5_polar))
dk['lats_array_era5_polar_nh']=(('months','years'),np.asarray(lats_array_era5_polar_nh))
dk['lats_array_era5_polar_sh']=(('months','years'),np.asarray(lats_array_era5_polar_sh))

dk['nums_array_era5_fixed']=(('months','years'),np.asarray(nums_array_era5_fixed))
dk['areas_array_era5_fixed']=(('months','years'),np.asarray(areas_array_era5_fixed))
dk['lats_array_era5_fixed']=(('months','years'),np.asarray(lats_array_era5_fixed))
dk['lats_array_era5_fixed_nh']=(('months','years'),np.asarray(lats_array_era5_fixed_nh))
dk['lats_array_era5_fixed_sh']=(('months','years'),np.asarray(lats_array_era5_fixed_sh))

dk['nums_array_era5_fixed_poleward']=(('months','years'),np.asarray(nums_array_era5_fixed_poleward))
dk['areas_array_era5_fixed_poleward']=(('months','years'),np.asarray(areas_array_era5_fixed_poleward))
dk['lats_array_era5_fixed_poleward']=(('months','years'),np.asarray(lats_array_era5_fixed_poleward))
dk['lats_array_era5_fixed_poleward_nh']=(('months','years'),np.asarray(lats_array_era5_fixed_poleward_nh))
dk['lats_array_era5_fixed_poleward_sh']=(('months','years'),np.asarray(lats_array_era5_fixed_poleward_sh))

dk['nums_list_era5_polar']=(('years'),np.asarray(nums_list_era5_polar))
dk['areas_list_era5_polar']=(('years'),np.asarray(areas_list_era5_polar))
dk['lats_list_era5_polar']=(('years'),np.asarray(lats_list_era5_polar))
dk['lats_list_era5_polar_nh']=(('years'),np.asarray(lats_list_era5_polar_nh))
dk['lats_list_era5_polar_sh']=(('years'),np.asarray(lats_list_era5_polar_sh))

dk['nums_list_era5_fixed']=(('years'),np.asarray(nums_list_era5_fixed))
dk['areas_list_era5_fixed']=(('years'),np.asarray(areas_list_era5_fixed))
dk['lats_list_era5_fixed']=(('years'),np.asarray(lats_list_era5_fixed))
dk['lats_list_era5_fixed_nh']=(('years'),np.asarray(lats_list_era5_fixed_nh))
dk['lats_list_era5_fixed_sh']=(('years'),np.asarray(lats_list_era5_fixed_sh))

dk['nums_list_era5_fixed_poleward']=(('years'),np.asarray(nums_list_era5_fixed_poleward))
dk['areas_list_era5_fixed_poleward']=(('years'),np.asarray(areas_list_era5_fixed_poleward))
dk['lats_list_era5_fixed_poleward']=(('years'),np.asarray(lats_list_era5_fixed_poleward))
dk['lats_list_era5_fixed_poleward_nh']=(('years'),np.asarray(lats_list_era5_fixed_poleward_nh))
dk['lats_list_era5_fixed_poleward_sh']=(('years'),np.asarray(lats_list_era5_fixed_poleward_sh))

dk.coords['years']=np.arange(1940,2024,1)
dk.coords['months']=np.arange(1,13,1)
try:
    os.remove(dir_data+'ar_chars_original_era5.nc')
except OSError:
    pass
dk.to_netcdf(dir_data+'ar_chars_original_era5.nc',mode='w',format='NETCDF4')